## UK's Online Retail Store Transaction Sales Analysis:

### Libraries Imported:

##### Pandas & Numpy - This Py_library used to Extract information about the data by manipulating it.
##### Matplotlib     - This library is used to show insightful info by visualizing the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

### Loading dataset - CSV {utf-8 coded}

In [2]:
df= pd.read_csv("Online_Retail_Store (UK).csv")

C:\Users\Ganesh kumar V\AppData\Local\Temp\ipykernel_10132\3712114491.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("Online_Retail_Store (UK).csv")


### Data Preprocessing

#### Understanding the Data

In [3]:
df.shape

(524878, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524878 entries, 0 to 524877
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   524878 non-null  int64  
 1   InvoiceNo    524878 non-null  object 
 2   StockCode    524878 non-null  object 
 3   Description  524878 non-null  object 
 4   Quantity     524878 non-null  int64  
 5   InvoiceDate  524878 non-null  object 
 6   UnitPrice    524878 non-null  float64
 7   CustomerID   392692 non-null  float64
 8   Country      524878 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 36.0+ MB


In [5]:
df.describe()

,Unnamed: 0,Quantity,UnitPrice,CustomerID
count,524878.000000,524878.000000,524878.000000,392692.000000
mean,271054.945892,10.616600,3.922573,15287.843865
std,156410.252156,156.280031,36.093028,1713.539549
min,0.000000,1.000000,0.001000,12346.000000
25%,135608.250000,1.000000,1.250000,13955.000000
50%,271132.000000,4.000000,2.080000,15150.000000
75%,406497.750000,11.000000,4.130000,16791.000000
max,541908.000000,80995.000000,13541.330000,18287.000000


#### From understanding the present data. It summarize that, 

- [Description, Customer ID] contains null values.
- [InvoiceDate]'s dtype need to be changed to datetime[ns].
- [Quantity, UnitPrice] Needs to check whether the values falls under 0. If yes it needs to be treated.

#### Standardizing the DateTime in InvoiceDate  

In [6]:
df["InvoiceDate"]= pd.to_datetime(df["InvoiceDate"],format="mixed",errors="coerce") 
#using format="mixed" cause the datetime present in mixed format  

In [7]:
df["InvoiceDate"].dtype

dtype('<M8[ns]')

In [8]:
df["InvoiceDate"].isna().sum()

np.int64(0)

In [9]:
df["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [10]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [11]:
df.sort_values("InvoiceDate").head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [12]:
df.sort_values("InvoiceDate").tail()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
524865,541896,581587,22555,PLASTERS IN TIN STRONGMAN,12,2011-12-09 12:50:00,1.65,12680.0,France
524864,541895,581587,22556,PLASTERS IN TIN CIRCUS PARADE,12,2011-12-09 12:50:00,1.65,12680.0,France
524876,541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
524869,541900,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680.0,France
524877,541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


#### Mandatory Check for Duplicates

In [13]:
df.duplicated().sum()

np.int64(0)

In [14]:
df.duplicated(keep=False).sum()

np.int64(0)

In [15]:
df[df.duplicated(keep=False)].groupby(df.columns.tolist()).size().value_counts()                 
#Shows how many times a row repaeated (A Random row repeated 20 Times)

Series([], Name: count, dtype: int64)

In [16]:
Total = (df["UnitPrice"]*df["Quantity"]).sum()
print("Total Revenue before Removing duplicates:", Total)

Total Revenue before Removing duplicates: 10642110.804000001


In [17]:
df = df.drop_duplicates()

In [18]:
df.duplicated().sum()
#Checking for any Duplicate had left over 

np.int64(0)

In [19]:
newtotal = (df["UnitPrice"]*df["Quantity"]).sum()
print("Total Revenue After Removing duplicates:", newtotal)

Total Revenue After Removing duplicates: 10642110.804000001


#### Treating the negative values

In [20]:
df[(df["Description"].isnull()) & 
   ((df["Quantity"] <= 0) | (df["UnitPrice"] <= 0))].shape

(0, 9)

#### Clarification for Nulls present in Description 
- The null values in the Description column are not random. All rows with missing descriptions also contain negative values in either UnitPrice or Quantity.
- These rows represent invalid or non-sale transactions that reduce the cleanliness of the dataset.
- While removing rows containing negative values in UnitPrice or Quantity, the rows with null descriptions were automatically removed as they belonged to the same invalid records.

In [21]:
df["Quantity"].min()

1

In [22]:
df["UnitPrice"].min()

0.001

In [23]:
invalidqty=(df["Quantity"]<=0).sum()
invalidup=(df["UnitPrice"]<=0).sum()

In [24]:
invalidqty

np.int64(0)

In [25]:
invalidup

np.int64(0)

In [26]:
df.shape    #Before removing

(524878, 9)

In [27]:
print("Total revenue before droping -ve values:",newtotal)

Total revenue before droping -ve values: 10642110.804000001


In [28]:
df = df[(df["Quantity"]>0)].copy()

In [29]:
df = df[(df["UnitPrice"]>0)].copy()

In [30]:
df.shape     #After removing

(524878, 9)

In [31]:
Uptotal=(df["Quantity"]*df["UnitPrice"]).sum()

In [32]:
print("Total revenue after droping -ve values:",Uptotal)

Total revenue after droping -ve values: 10642110.804000001


In [33]:
df["Quantity"].min()

1

In [34]:
df["UnitPrice"].min()

0.001

#### Checking for Null Values

In [35]:
df["Description"].isnull().sum()
# Nulls had been removed while treating the Negative Value.

np.int64(0)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524878 entries, 0 to 524877
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   524878 non-null  int64         
 1   InvoiceNo    524878 non-null  object        
 2   StockCode    524878 non-null  object        
 3   Description  524878 non-null  object        
 4   Quantity     524878 non-null  int64         
 5   InvoiceDate  524878 non-null  datetime64[ns]
 6   UnitPrice    524878 non-null  float64       
 7   CustomerID   392692 non-null  float64       
 8   Country      524878 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 36.0+ MB


In [37]:
Nv=df["CustomerID"].isnull()
print(Nv)

0         False
1         False
2         False
3         False
4         False
          ...  
524873    False
524874    False
524875    False
524876    False
524877    False
Name: CustomerID, Length: 524878, dtype: bool


In [38]:
print("Nulls Present in CustomerID:",Nv.sum())

Nulls Present in CustomerID: 132186


#### Fixing Nulls

#### Description = Text based Outcome, null = 0.27% [Missing]
- These were retained with a standerized placeholder to preserve revenue accuracy.

In [39]:
df["Description"]= df["Description"].fillna("Unkown Product")
df.isnull().sum()

Unnamed: 0          0
InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132186
Country             0
dtype: int64

#### CustomerID ~25% transactions were anonymous [Missing]. 
- If in case of sales analysis & generating revenue, it will be retained --- Transaction level.
- On the other, for Customer based analysis it should be excluded  --- Customer level.
- So Categorizing the dataset seperatly based on Customer Analysis & Sales Analysis.

In [40]:
df_transaction = df.drop(columns=["CustomerID"])
df_customer = df[df["CustomerID"].notna()].copy()

#### Checking the categorized DataFrame 
- Wheather it Contains Appropriate dtype and containing No Null values

In [41]:
df_transaction.isnull().sum()

Unnamed: 0     0
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
Country        0
dtype: int64

In [42]:
df_customer.isnull().sum()

Unnamed: 0     0
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [43]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524878 entries, 0 to 524877
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   524878 non-null  int64         
 1   InvoiceNo    524878 non-null  object        
 2   StockCode    524878 non-null  object        
 3   Description  524878 non-null  object        
 4   Quantity     524878 non-null  int64         
 5   InvoiceDate  524878 non-null  datetime64[ns]
 6   UnitPrice    524878 non-null  float64       
 7   Country      524878 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 32.0+ MB


In [44]:
df_customer["CustomerID"] = df_customer["CustomerID"].astype(int)

In [45]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392692 entries, 0 to 524877
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   392692 non-null  int64         
 1   InvoiceNo    392692 non-null  object        
 2   StockCode    392692 non-null  object        
 3   Description  392692 non-null  object        
 4   Quantity     392692 non-null  int64         
 5   InvoiceDate  392692 non-null  datetime64[ns]
 6   UnitPrice    392692 non-null  float64       
 7   CustomerID   392692 non-null  int64         
 8   Country      392692 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 30.0+ MB


### Assumptions and Scope of Analysis

This analysis focuses exclusively on valid sales transactions. Records containing non-positive values in Quantity or UnitPrice were excluded, as these represent cancellations, returns, or accounting adjustment entries rather than actual sales activity.

Exact duplicate transaction rows were identified and removed to prevent the revenue metrics. Invoice dates were standardized to a datetime format to enable accurate time-based analysis.

Missing product descriptions were found to be associated with invalid transactions and were removed alongside those records. Customer identifiers were not imputed; instead, the dataset was separated into transaction-level and customer-level views to ensure accurate customer analysis while preserving overall revenue integrity.

The resulting dataset reflects true commercial sales activity and is suitable for KPI reporting and business insight generation.

### Conclusion

##### Done with the Data Preprocessing steps.
- Moving forward by building the sales dashboard for the transaction level Analysis by using Power BI.

### Calculating KPIs 

In [46]:
df_transaction["Revenue"] = df_transaction["Quantity"]*df_transaction["UnitPrice"]

In [47]:
total_revenue=df_transaction["Revenue"].sum()

In [48]:
total_revenue

np.float64(10642110.804000001)

In [49]:
total_orders=df_transaction["InvoiceNo"].nunique()

In [50]:
total_orders

19962

In [51]:
AOV= total_revenue/total_orders
#Average order value

In [52]:
AOV

np.float64(533.1184652840398)

In [53]:
TotalQuanity_sold = df_transaction["Quantity"].sum()

In [54]:
TotalQuanity_sold

np.int64(5572420)

In [55]:
df_transaction["InvoiceDate"].min()   

Timestamp('2010-12-01 08:26:00')

In [56]:
df_transaction["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

### Exporting DF's[Transaction & Customer]

In [57]:
df_transaction.to_csv("transaction_clean.csv",index=True)

In [58]:
df_customer.to_csv("customer_clean.csv",index=True)

In [59]:
df.to_csv("Online_Retail_Store (UK) Clean.csv",index=True)